In [92]:
import ast

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [93]:
df = pd.read_csv('data/DTP_DATA_2025.csv')
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
features_number = df.shape[1]
df.head(5)

C:\Users\Настя\AppData\Local\Temp\ipykernel_22752\1620449790.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/DTP_DATA_2025.csv')


(1475674, 59)


,REGION,TYPE,DATE,traffic_changes,COORD_L,COORD_W,road_name,road_category,road_rank,cause_factors,...,n_front_drive,n_rear_drive,n_4wd,n_guilty,guilty_share,n_fatal_violations,violations,injury_severity,guilty_exp_avg,exp_avg
0,1,Столкновение,31.01.2015,Движение частично перекрыто,83.740833,53.333056,NaN,NaN,Не указано,Сведения отсутствуют,...,0,0,0,1,0.5,1,{'Другие нарушения ПДД водителем'},"{'Раненый, находящийся (находившийся) на амбу...",15.0,13.0
1,1,Наезд на пешехода,31.01.2015,Режим движения не изменялся,82.462778,52.245833,А-322 Барнаул - Рубцовск - граница с Республик...,3.0,Федеральная (дорога федерального значения),Сведения отсутствуют,...,0,0,0,1,1.0,2,"{'Несоблюдение требований ОСАГО', 'Нахождение ...","{'Раненый, находящийся (находившийся) на стаци...",NaN,5.0
2,1,Наезд на пешехода,31.01.2015,Движение частично перекрыто,83.501944,53.274167,NaN,NaN,Не указано,Сведения отсутствуют,...,0,0,0,2,2.0,1,{'Переход через проезжую часть вне пешеходного...,"{'Раненый, находящийся (находившийся) на амбу...",NaN,17.0
3,1,Наезд на пешехода,31.01.2015,Режим движения не изменялся,83.251667,53.285278,NaN,NaN,Не указано,Сведения отсутствуют,...,0,0,0,1,1.0,1,{'Переход через проезжую часть вне пешеходного...,"{'Раненый, находящийся (находившийся) на амбу...",NaN,27.0
4,1,Наезд на пешехода,31.01.2015,Движение частично перекрыто,83.708889,53.318333,NaN,NaN,Не указано,Сведения отсутствуют,...,0,0,0,1,1.0,1,{'Переход через проезжую часть в неустановленн...,"{'Раненый, находящийся (находившийся) на стаци...",NaN,1.0


In [94]:
df.columns

Index(['REGION', 'TYPE', 'DATE', 'traffic_changes', 'COORD_L', 'COORD_W',
       'road_name', 'road_category', 'road_rank', 'cause_factors',
       'street_rank', 'road_km', 'road_m', 'NP', 'road_defects', 'adj_objects',
       'lighting', 'road_surface', 'site_objects', 'weather', 'n_VEHICLES',
       'n_PARTICIPANTS', 'ID', 'n_DEATHS', 'n_INJURED', 'TIME',
       'vehicle_failure', 'non_private_vehicle', 'russian_vehicle',
       'white_vehicle', 'black_vehicle', 'colored_vehicle', 'drunk_driver',
       'female_driver', 'escaped', 'no_seatbelt_injury', 'n_drunk',
       'n_children', 'n_cyclists', 'n_pedestrians', 'vehicle_age_min',
       'vehicle_age_max', 'vehicle_age_avg', 'n_class_a', 'n_class_b',
       'n_class_c', 'n_class_d', 'n_class_e', 'n_class_s', 'n_front_drive',
       'n_rear_drive', 'n_4wd', 'n_guilty', 'guilty_share',
       'n_fatal_violations', 'violations', 'injury_severity', 'guilty_exp_avg',
       'exp_avg'],
      dtype='object')

In [95]:
print(f"Number of observations before processing: {df.shape[0]}")

Number of observations before processing: 1475674


## Processing the data gaps

In [96]:
features_with_gaps = df.columns[df.isna().any()].tolist()

for i, col in enumerate(features_with_gaps, start=1):
    print(f"{i}. {col}")

1. traffic_changes
2. COORD_L
3. COORD_W
4. road_name
5. road_category
6. street_rank
7. road_km
8. road_m
9. NP
10. vehicle_age_min
11. vehicle_age_max
12. vehicle_age_avg
13. guilty_exp_avg


We don't need every feature, so let's create a list of columns to drop and update it as needed.

In [97]:
drop_columns = ['road_km', 'road_m']
# will be dropped because they are irrelevant to our task

__traffic_changes__: 2 objects were dropped

In [98]:
print(f"traffic_changes gaps: {df['traffic_changes'].isna().sum()}")

df = df[df['traffic_changes'].notna()]
print(f"Objects left: {df.shape[0]}")

traffic_changes gaps: 2
Objects left: 1475672


__COORD_L and COORD_W:__

In [99]:
df[['COORD_L', 'COORD_W']].dtypes

# turn into float64
df['COORD_L'] = pd.to_numeric(df['COORD_L'], errors='coerce')
df['COORD_W'] = pd.to_numeric(df['COORD_W'], errors='coerce')

df[['COORD_L', 'COORD_W']].dtypes

COORD_L    float64
COORD_W    float64
dtype: object

Begin with analyzing columns gaps: 15 objects were dropped

In [100]:
L_ind = set(df[df['COORD_L'].isna()]['ID'])
W_ind = set(df[df['COORD_W'].isna()]['ID'])
common_gaps = L_ind | W_ind

print(f"COORD_L gaps: {df['COORD_L'].isna().sum()}")
print(f"COORD_W gaps: {df['COORD_W'].isna().sum()}")
print(f"COORD_L and COORD_W common gaps: {len(common_gaps)}")

df = df[(df['COORD_L'].notna()) & (df['COORD_W'].notna())]
print(f"COORD_L gaps: {df['COORD_L'].isna().sum()}")
print(f"COORD_W gaps: {df['COORD_W'].isna().sum()}")
print(f"Objects left: {df.shape[0]}")

COORD_L gaps: 166
COORD_W gaps: 156
COORD_L and COORD_W common gaps: 167
COORD_L gaps: 0
COORD_W gaps: 0
Objects left: 1475505


### **road_name, road_category:** key features in our analysis

Why `road_name` is a key feature?
- contains road names
- will allow to identify accidents on toll roads (here you can find [the list of toll roads in Russia](https://avtodor-tr.ru/road/))
- so we can create binary key feature `toll road`

In [101]:
print(f"Gaps in road_name: {df['road_name'].isna().sum()}")
print(f"Gaps in road_category: {df['road_category'].isna().sum()}")

dor_ind = set(df[df['road_name'].isna()]['ID'])
dor_k_ind = set(df[df['road_category'].isna()]['ID'])
common_gaps = dor_ind | dor_k_ind

print(f"Road_name and road_category common gaps: {len(common_gaps)}")

Gaps in road_name: 964910
Gaps in road_category: 971592
Road_name and road_category common gaps: 977475


In [102]:
# now we are not going to fill the gaps (do it later), so drop NaN

df = df[df['road_name'].notna()]
print(f"Observations left after deleting NaN in 'road_name' feature: {df.shape[0]}")

Observations left after deleting NaN in 'road_name' feature: 510595


__street_rank:__ replace nan by new value

In [103]:
print(f"street_rank gaps: {df['street_rank'].isna().sum()}")

df['street_rank'] = df['street_rank'].fillna('Назначение неизвестно')
print(f"street_rank gaps left: {df['street_rank'].isna().sum()}")

street_rank gaps: 127857
street_rank gaps left: 0


__guilty_exp_avg:__ 145498 objects were deleted

In [104]:
print(f"guilty_exp_avg gaps: {df['guilty_exp_avg'].isna().sum()}")

df = df[df['guilty_exp_avg'].notna()]
print(f"Objects left: {df.shape[0]}")

guilty_exp_avg gaps: 37401
Objects left: 473194


__vehicle_age_min, vehicle_age_max, vehicle_age_avg:__ will be filled by ML algorithm later

In [105]:
print(f"Gaps in vehicle_age_min: {df['vehicle_age_min'].isna().sum()}")
print(f"Gaps in vehicle_age_max: {df['vehicle_age_max'].isna().sum()}")
print(f"Gaps in vehicle_age_avg: {df['vehicle_age_avg'].isna().sum()}")

min_ind = set(df[df['vehicle_age_min'].isna()]['ID'])
max_ind = set(df[df['vehicle_age_max'].isna()]['ID'])
avg_ind = set(df[df['vehicle_age_avg'].isna()]['ID'])
common_gaps = min_ind | max_ind | avg_ind

print(f"Common gaps: {len(common_gaps)}")

Gaps in vehicle_age_min: 6208
Gaps in vehicle_age_max: 6208
Gaps in vehicle_age_avg: 6208
Common gaps: 6208


Wi will train a model that will predict missing values ​​based on other features below.

#### **Lyrical digression and NP:** why it is useless and will be deleted

In [106]:
print(f"NP gaps: {df['NP'].isna().sum()}")

NP gaps: 358663


`NP` contains only (compared to `dor`) 388.721 gaps and had a potential to be a key feature.

Why `NP` could be a key feature?
- the aim of the thesis is not only to analyze toll roads but also to compare them with free roads in Russia
- toll roads in Russia are mostly highways, which makes them very different from accidents occurring within cities (due to different conditions, speed limits, etc.)
- for a correct analysis, accidents that occurred in populated areas must be excluded from the DataFrame
- identifying these areas using NP could help achieve this

Why it is NOT?
- because the feature doesn't work tha way it should
- according to its intended meaning, `NP` should either: 

(1) contain names of populated areas with non-populated areas represented as NaN or 

(2) include names of populated areas and a clear label for non-populated areas (e.g., "not NP" or "highway")

- in reality, `NP` contains both names of populated areas and labels for non-populated areas, but these labels account for less than 0.2% of the non-NaN observations in `NP`
- as a result, the column primarily reflects the names of populated areas nearest to the accident location — even when the accident occurred on a highway
- this makes the feature completely useless for the analysis objectives, as it does not reliably distinguish between accidents in populated and non-populated areas

In [107]:
np_marks_list = [
    'г ', 'д ', 'п ', 'с ', 'х ', 'ст-ца ', 'аал ', 'аул ', 'массив ', 'мкр ', 
    'пгт ', 'починок ', 'сл ', 'ст ', 'поселение ', 'жилрайон ', 'городок ', 'заимка ', 'м ', 'у'
    ]           # compiled by the method of analyzing unique 'NP' feature values

non_missing_np = df[df['NP'].notna()]
not_np_unique = non_missing_np['NP'].unique().tolist()
not_np_unique = sorted(not_np_unique)

for np in not_np_unique[:]:
    for np_mark in np_marks_list:
        if np_mark in np:
            not_np_unique.remove(np)
            break

non_NP_obs = df[df['NP'].isin(not_np_unique)]
print(f"Number of observations without gaps in 'NP', where 'NP' values are not considered as populated areas: {non_NP_obs.shape[0]}")
print('-----')
print(' ')

# and if we try to find any highway in this DataFrame we'll fail:

dor_uniq = df['road_name'].unique().tolist()
M_1_dors = list()

for dor in dor_uniq:
    try:
        if 'М-1 ' in dor and 'подъездная' not in dor and 'Подъездная' not in dor:
            M_1_dors.append(dor)
    except TypeError:
        continue

M1_df = non_NP_obs[non_NP_obs['road_name'].isin(M_1_dors)]
print(f"M-1 highway found in non_NP_obs DataFrame: {M1_df.shape[0]}")
print('Spoiler: the number will be the same for every toll road')
print('-----')
print(' ')

# and in full DataFrame:

M1_df = df[df['road_name'].isin(M_1_dors)]
print(f"M-1 highway found in full DataFrame: {M1_df.shape[0]}")
print('-----')
print(' ')

# analyzing the 'NP' values of M-1 toll road observations and finding out the 'NP' uselesness

print('The M-1 toll road unique NP values (M-1 road is a highway):')
M1_df[M1_df['NP'].notna()]['NP'].unique()

Number of observations without gaps in 'NP', where 'NP' values are not considered as populated areas: 275
-----
 
M-1 highway found in non_NP_obs DataFrame: 0
Spoiler: the number will be the same for every toll road
-----
 
M-1 highway found in full DataFrame: 1642
-----
 
The M-1 toll road unique NP values (M-1 road is a highway):


array(['г Москва', 'г Кубинка', 'д Ляхово', 'с/п Новоивановское',
       'с/п Успенское', 'рп Новоивановское', 'д Барвиха', 'д Бутынь',
       'г Одинцово', 'г Клин', 'дп Лесной Городок', 'д Немчиново',
       'д Вырубово', 'с Жаворонки', 'д Красная Горка', 'д Каменка',
       'г Ярцево', 'д Анохово', 'г Гагарин'], dtype=object)

In [108]:
drop_columns.append('NP')

## Processing categorical features

Which features are categorical?

In [109]:
cat_cols = df.select_dtypes(include=['object']).columns
non_cat_cols = ['COORD_L', 'COORD_W', 'DATE', 'TIME']
non_cat_cols.append('road_name')      # road names will be necessary on further steps
cat_cols = list(set(cat_cols) - set(drop_columns) - set(non_cat_cols))      # do not proceed unnecessary features

for i, col in enumerate(cat_cols, start=1):
    print(f"{i}. {col}")

1. injury_severity
2. cause_factors
3. street_rank
4. road_defects
5. TYPE
6. traffic_changes
7. site_objects
8. violations
9. weather
10. road_rank
11. road_surface
12. lighting
13. adj_objects


We will handle each categorical feature manually to ensure consistent labeling and avoid issues caused by duplicates, typos, or overly detailed values that should be grouped together.

---

__road_rank — road significance__

In [110]:
sorted(df['road_rank'].unique())

['Другие места',
 'Иная дорога',
 'Местного значения (дорога местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)',
 'Местного значения (дороги местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)',
 'Не указано',
 'Региональная или межмуниципальная (дорога регионального или межмуниципального значения)',
 'Федеральная (дорога федерального значения)',
 'Частная (дорога, относящиеся к частной и иным формам собственности)',
 'Частная (дороги, относящиеся к частной и иным формам собственности)']

In [111]:
replace_map = {
    'Региональная или межмуниципальная (дорога регионального или межмуниципального значения)': 'Региональная или межмуниципальная',
    'Федеральная (дорога федерального значения)': 'Федеральная',
    'Местного значения (дороги местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)': 'Местного значения',
    'Местного значения (дорога местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)': 'Местного значения',
    'Частная (дороги, относящиеся к частной и иным формам собственности)': 'Частная',
    'Частная (дорога, относящиеся к частной и иным формам собственности)': 'Частная',
    'Другие места': 'Другое',
    'Иная дорога': 'Другое',
    'Не указано': 'Не указано'
}

df['road_rank'] = df['road_rank'].replace(replace_map)
sorted(df['road_rank'].unique())

['Другое',
 'Местного значения',
 'Не указано',
 'Региональная или межмуниципальная',
 'Федеральная',
 'Частная']

In [112]:
le = LabelEncoder()
df['road_rank_cat'] = le.fit_transform(df['road_rank'])

---

**road_defects — defects of the road network transport and operational maintenance**

In [113]:
sorted(df['road_defects'].unique())

['Дефекты покрытия',
 'Иные недостатки',
 'Нарушения в размещении наружной рекламы',
 'Не установлены',
 'Недостатки зимнего содержания',
 'Недостаточное освещение',
 'Неисправное освещение',
 'Неисправность светофора',
 'Неправильное применение, плохая видимость дорожных знаков',
 'Неровное покрытие',
 'Несоответствие дорожных ограждений предъявляемым требованиям',
 'Несоответствие железнодорожного переезда предъявляемым требованиям',
 'Несоответствие люков смотровых колодцев и ливневой канализации предъявляемым требованиям',
 'Неудовлетворительное состояние обочин',
 'Неудовлетворительное состояние разделительной полосы',
 'Низкие сцепные качества покрытия',
 'Ограничение видимости',
 'Отклонение верха головки рельса трамвайных (железнодорожных) путей, расположенных в пределах проезжей части, относительно покрытия, более чем на 2,0 см',
 'Отсутствие временных ТСОД в местах проведения работ',
 'Отсутствие дорожных знаков в необходимых местах',
 'Отсутствие дорожных ограждений в необхо

In [114]:
replace_map = {
    # Lighting and traffic lights
    'Недостаточное освещение': 'Освещение',
    'Неисправное освещение': 'Освещение',
    'Отсутствие освещения': 'Освещение',
    'Плохая видимость светофора': 'Освещение',
    'Неисправность светофора': 'Освещение',

    # Pavement and roadway
    'Дефекты покрытия': 'Покрытие',
    'Неровное покрытие': 'Покрытие',
    'Низкие сцепные качества покрытия': 'Покрытие',
    'Сужение проезжей части, наличие препятствий, затрудняющих движение транспортных средств': 'Покрытие',
    'Отклонение верха головки рельса трамвайных (железнодорожных) путей, расположенных в пределах проезжей части, относительно покрытия, более чем на 2,0 см': 'Покрытие',

    # Markup
    'Отсутствие, плохая различимость вертикальной разметки': 'Разметка',
    'Отсутствие, плохая различимость горизонтальной разметки проезжей части': 'Разметка',

    # Signs
    'Отсутствие дорожных знаков в необходимых местах': 'Дорожные знаки',
    'Неправильное применение, плохая видимость дорожных знаков': 'Дорожные знаки',

    # Fences and devices
    'Отсутствие дорожных ограждений в необходимых местах': 'Ограждения',
    'Отсутствие пешеходных ограждений в необходимых местах': 'Ограждения',
    'Плохая видимость световозвращателей, размещенных на дорожных ограждениях': 'Ограждения',
    'Отсутствие направляющих устройств и световозвращающих элементов на них': 'Ограждения',

    # Sidewalks, bus stops
    'Отсутствие тротуаров (пешеходных дорожек)': 'Пешеходная инфраструктура',
    'Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта': 'Пешеходная инфраструктура',

    # Road shoulders and median strips
    'Неудовлетворительное состояние обочин': 'Обочины и разделительные полосы',
    'Неудовлетворительное состояние разделительной полосы': 'Обочины и разделительные полосы',

    # Railway, manholes, crossings
    'Несоответствие железнодорожного переезда предъявляемым требованиям': 'Переезды и ж/д инфраструктура',
    'Несоответствие люков смотровых колодцев и ливневой канализации предъявляемым требованиям': 'Люки и колодцы',
    
    # Other
    'Отсутствие временных ТСОД в местах проведения работ': 'Отсутствие ТСОД',
    'Нарушения в размещении наружной рекламы': 'Наружная реклама',
    'Недостатки зимнего содержания': 'Зимнее содержание',
    'Несоответствие дорожных ограждений предъявляемым требованиям': 'Ограждения',
    'Ограничение видимости': 'Видимость',
    'Иные недостатки': 'Прочее',
    'Не установлены': 'Не указано'
}

df['road_defects'] = df['road_defects'].replace(replace_map)
sorted(df['road_defects'].unique())

['Видимость',
 'Дорожные знаки',
 'Зимнее содержание',
 'Люки и колодцы',
 'Наружная реклама',
 'Не указано',
 'Обочины и разделительные полосы',
 'Ограждения',
 'Освещение',
 'Отсутствие ТСОД',
 'Переезды и ж/д инфраструктура',
 'Пешеходная инфраструктура',
 'Покрытие',
 'Прочее',
 'Разметка']

In [115]:
le = LabelEncoder()
df['road_defects_cat'] = le.fit_transform(df['road_defects'])

---
**traffic_changes — info about changes in traffic**

In [116]:
sorted(df['traffic_changes'].unique())

['Движение полностью перекрыто',
 'Движение частично перекрыто',
 'Режим движения не изменялся']

In [117]:
def get_changes_in_traffic(category):
    if category in ['Движение частично перекрыто', 'Движение полностью перекрыто']:
        return 1
    else:
        return 0

df['traffic_changes_bin'] = df['traffic_changes'].apply(get_changes_in_traffic)
sorted(df['traffic_changes_bin'].unique())

[0, 1]

In [118]:
le = LabelEncoder()
df['traffic_changes_cat'] = le.fit_transform(df['traffic_changes'])

---
**road_surface — road surface condition**

In [119]:
sorted(df['road_surface'].unique())

['Гололедица',
 'Загрязненное',
 'Залитое (покрытое) водой',
 'Заснеженное',
 'Мокрое',
 'Не установлено',
 'Обработанное противогололедными материалами',
 'Пыльное',
 'Свежеуложенная поверхностная обработка',
 'Со снежным накатом',
 'Сухое']

In [120]:
replace_map = {
    'Сухое': 'Сухое',
    
    'Мокрое': 'Мокрое',
    'Залитое (покрытое) водой': 'Мокрое',
    
    'Заснеженное': 'Заснеженное',
    'Со снежным накатом': 'Заснеженное',
    'Гололедица': 'Гололед',
    
    'Обработанное противогололедными материалами': 'Обработанное',
    
    'Пыльное': 'Пыльное',
    'Загрязненное': 'Пыльное',
    
    'Свежеуложенная поверхностная обработка': 'Обработка',
    
    'Не установлено': 'Не указано'
}

df['road_surface'] = df['road_surface'].replace(replace_map)
sorted(df['road_surface'].unique())

['Гололед',
 'Заснеженное',
 'Мокрое',
 'Не указано',
 'Обработанное',
 'Обработка',
 'Пыльное',
 'Сухое']

In [121]:
le = LabelEncoder()
df['road_surface_cat'] = le.fit_transform(df['road_surface'])

---

__TYPE__

In [122]:
sorted(df['TYPE'].unique())

['Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.',
 'Иной вид ДТП',
 'Наезд на велосипедиста',
 'Наезд на внезапно возникшее препятствие',
 'Наезд на гужевой транспорт',
 'Наезд на животное',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ',
 'Наезд на пешехода',
 'Наезд на препятствие',
 'Наезд на стоящее ТС',
 'Опрокидывание',
 'Отбрасывание предмета',
 'Падение груза',
 'Падение пассажира',
 'Столкновение',
 'Съезд с дороги']

In [123]:
replace_map = {
    # Collisions
    'Столкновение': 'Столкновение',
    'Опрокидывание': 'Столкновение',
    'Съезд с дороги': 'Столкновение',

    # Collisions with pedestrians and people
    'Наезд на пешехода': 'Наезд на пешехода',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность': 'Наезд на человека',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы': 'Наезд на человека',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ': 'Наезд на человека',
    'Наезд на велосипедиста': 'Наезд на велосипедиста',

    # Collisions with objects and obstacles
    'Наезд на препятствие': 'Наезд на препятствие',
    'Наезд на стоящее ТС': 'Наезд на препятствие',
    'Наезд на внезапно возникшее препятствие': 'Наезд на препятствие',

    # Other raids
    'Наезд на животное': 'Наезд на животное',
    'Наезд на гужевой транспорт': 'Наезд на животное',

    # Other
    'Падение пассажира': 'Прочее',
    'Падение груза': 'Прочее',
    'Отбрасывание предмета': 'Прочее',
    'Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.': 'Прочее',
    
    'Иной вид ДТП': 'Другое'
}

df['TYPE'] = df['TYPE'].replace(replace_map)
sorted(df['TYPE'].unique())

['Другое',
 'Наезд на велосипедиста',
 'Наезд на животное',
 'Наезд на пешехода',
 'Наезд на препятствие',
 'Наезд на человека',
 'Прочее',
 'Столкновение']

In [124]:
le = LabelEncoder()
df['TYPE_cat'] = le.fit_transform(df['TYPE'])

--- 

__street_rank — street significance__

In [125]:
sorted(df['street_rank'].unique())

['Велосипедные дорожки',
 'Вне НП',
 'Второстепенные улицы в жилой застройке (переулки)',
 'Главные улицы',
 'Иные места',
 'Магистральные дороги',
 'Магистральные улицы общегородского значения',
 'Магистральные улицы районного значения',
 'Назначение неизвестно',
 'Основные улицы',
 'Основные улицы в жилой застройке',
 'Парковые дороги',
 'Парковые дороги, пешеходные улицы (пешеходные зоны)',
 'Пешеходные улицы',
 'Поселковые дороги',
 'Проезды',
 'Улицы и дороги местного значения в жилой застройке',
 'Улицы и дороги местного значения в производственных, промышленных и коммунально-складских зонах',
 'Улицы и дороги местного значения научно-производственных промышленных и коммунально-складских районов',
 'Хозяйственные проезды, скотопрогоны']

In [126]:
replace_map = {
    # Highways and main streets
    'Магистральные дороги': 'Магистральная дорога',
    
    'Магистральные улицы общегородского значения': 'Магистральная улица',
    'Магистральные улицы районного значения': 'Магистральная улица',
    'Главные улицы': 'Магистральная улица',
    'Основные улицы': 'Магистральная улица',
    'Основные улицы в жилой застройке': 'Магистральная улица',

    # Local streets
    'Второстепенные улицы в жилой застройке (переулки)': 'Улица местного значения',
    'Улицы и дороги местного значения в жилой застройке': 'Улица местного значения',
    'Улицы и дороги местного значения в производственных, промышленных и коммунально-складских зонах': 'Улица местного значения',
    'Улицы и дороги местного значения научно-производственных промышленных и коммунально-складских районов': 'Улица местного значения',
    'Проезды': 'Улица местного значения',
    'Поселковые дороги': 'Улица местного значения',
    'Хозяйственные проезды, скотопрогоны': 'Улица местного значения',

    # Pedestrian areas and parks
    'Пешеходные улицы': 'Пешеходная зона',
    'Парковые дороги': 'Пешеходная зона',
    'Парковые дороги, пешеходные улицы (пешеходные зоны)': 'Пешеходная зона',
    'Велосипедные дорожки': 'Пешеходная зона',

    # Other
    'Вне НП': 'Вне НП',
    'Назначение неизвестно': 'Не указано',
    'Иные места': 'Другое'
}

df['street_rank'] = df['street_rank'].replace(replace_map)
sorted(df['street_rank'].unique())

['Вне НП',
 'Другое',
 'Магистральная дорога',
 'Магистральная улица',
 'Не указано',
 'Пешеходная зона',
 'Улица местного значения']

To create the final datasets in our task, we need to filter out observations that occurred outside the city limits, as accidents in populated areas happen under completely different conditions and cannot be compared to those on toll roads.

As we know the variable `NP`, or populated area, turned out to be completely useless because it showed the nearest populated area to the accident, rather than whether the accident occurred in a populated area or not. However, we still have the `street_rank` variable, which accurately reflects where the accident occurred. We'll extract this information into a separate binary feature `out_of_town` before transforming `street_rank` into a categorical variable, since its original meaning will no longer be accessible after encoding.

In [127]:
df['out_of_town'] = df['street_rank'].isin([
    'Вне НП',
    'Не указано',
    'Магистральная дорога'
]).astype(int)

In [128]:
le = LabelEncoder()
df['street_rank_cat'] = le.fit_transform(df['street_rank'])

---

__weather__

In [129]:
sorted(df['weather'].unique())

['Дождь',
 'Метель',
 'Пасмурно',
 'Снегопад',
 'Температура выше +30С',
 'Температура ниже -30С',
 'Туман',
 'Ураганный ветер',
 'Ясно']

In [130]:
def get_weather(category):
    if category in ['Снегопад', 'Метель']:
        return 'SMOW'
    elif category in ['Туман', 'Пасмурно']:
        return 'LOW_VISIBILITY'
    elif category == 'Температура выше +30С':
        return 'ABOVE_30_DGR'
    elif category == 'Температура ниже -30С':
        return 'BELOW_30_DGR'
    elif category == 'Ураганный ветер':
        return 'HURRICANE_WIND'
    else:
        return 'nothing'

df['weather_interpretable'] = df['weather'].apply(get_weather)
df['weather_interpretable'].unique()

array(['nothing', 'LOW_VISIBILITY', 'SMOW', 'HURRICANE_WIND',
       'ABOVE_30_DGR', 'BELOW_30_DGR'], dtype=object)

In [131]:
replace_map = {
    # Precipitation
    'Дождь': 'Осадки',
    'Снегопад': 'Осадки',
    'Метель': 'Осадки',

    # Poor visibility
    'Туман': 'Плохая видимость',
    'Пасмурно': 'Плохая видимость',

    # Extreme temperatures
    'Температура выше +30С': 'Жара',
    'Температура ниже -30С': 'Мороз',

    # Wind
    'Ураганный ветер': 'Сильный ветер',

    # Good weather
    'Ясно': 'Хорошая погода'
}

df['weather'] = df['weather'].replace(replace_map)
sorted(df['weather'].unique())

['Жара',
 'Мороз',
 'Осадки',
 'Плохая видимость',
 'Сильный ветер',
 'Хорошая погода']

In [132]:
le = LabelEncoder()
df['weather_cat'] = le.fit_transform(df['weather'])

---

__adj_objects — road network objects near RTA__

In [133]:
sorted(df['adj_objects'].unique())

['АЗС',
 'Автовокзал (автостанция)',
 'Автостоянка (отделенная от проезжей части)',
 'Административные здания',
 'Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань)',
 'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)',
 'Жилые дома индивидуальной застройки',
 'Зоны отдыха',
 'Иная образовательная организация',
 'Иное образовательное учреждение',
 'Иной объект',
 'Кладбище',
 'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта)',
 'Ледовая переправа',
 'Ледовая переправа (официально открытая и оборудованная);',
 'Лечебные учреждения',
 'Медицинские (лечебные) организации',
 'Многоквартирные жилые дома',
 'Мост',
 'Надземный пешеходный переход',
 'Нерегулируемое пересечение с круговым движением',
 'Нерегулируемый ж/д переезд',
 'Нерегулируемый перекрёсток',
 'Нерегулируемый пешеходный переход',
 'Объект (здание, сооружение) религиозного культа',
 'Объект строительст

In [134]:
df['road_name'].nunique()

24722

In [135]:
replace_map = {
    # Education
    'Школа либо иная детская (в т.ч. дошкольная) организация': 'Образование',
    'Школа либо иное детское (в т.ч. дошкольное) учреждение': 'Образование',
    'Иная образовательная организация': 'Образование',
    'Иное образовательное учреждение': 'Образование',

    # Medicine
    'Лечебные учреждения': 'Медицина',
    'Медицинские (лечебные) организации': 'Медицина',

    # Residential development
    'Жилые дома индивидуальной застройки': 'Жилая застройка',
    'Многоквартирные жилые дома': 'Жилая застройка',

    # Trade and food
    'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта)': 'Торговля',
    'Одиночный торговый объект, являющийся местом притяжения транспорта и (или) пешеходов': 'Торговля',
    'Объект торговли, общественного питания на автодороге вне НП': 'Торговля',

    # Transport facilities
    'АЗС': 'Транспортная инфраструктура',
    'Автовокзал (автостанция)': 'Транспортная инфраструктура',
    'Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань)': 'Транспортная инфраструктура',
    'Остановка маршрутного такси': 'Остановка',
    'Остановка общественного транспорта': 'Остановка',
    'Остановка трамвая': 'Остановка',
    'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)': 'Транспортная инфраструктура',
    'Автостоянка (отделенная от проезжей части)': 'Транспортная инфраструктура',
    'Пункт весового контроля': 'Транспортная инфраструктура',

    # Road infrastructure
    'Регулируемый перекрёсток': 'Дорожная инфраструктура',
    'Нерегулируемый перекрёсток': 'Дорожная инфраструктура',
    'Нерегулируемое пересечение с круговым движением': 'Дорожная инфраструктура',
    'Регулируемый пешеходный переход': 'Дорожная инфраструктура',
    'Нерегулируемый пешеходный переход': 'Дорожная инфраструктура',
    'Подземный пешеходный переход': 'Дорожная инфраструктура',
    'Надземный пешеходный переход': 'Дорожная инфраструктура',
    'Регулируемый ж/д переезд с дежурным': 'Дорожная инфраструктура',
    'Регулируемый ж/д переезд без дежурного': 'Дорожная инфраструктура',
    'Нерегулируемый ж/д переезд': 'Дорожная инфраструктура',
    'Мост': 'Дорожная инфраструктура',
    'Эстакада, путепровод': 'Дорожная инфраструктура',
    'Тоннель': 'Дорожная инфраструктура',
    'СП ДПС (КПМ)': 'Дорожная инфраструктура',

    # Other
    'Объект (здание, сооружение) религиозного культа': 'Прочее',
    'Административные здания': 'Прочее',
    'Зоны отдыха': 'Прочее',
    'Кладбище': 'Прочее',
    'Объект строительства': 'Прочее',
    'Производственное предприятие': 'Прочее',
    'Территориальное подразделение МВД России (либо его структурное подразделение)': 'Прочее',
    'Спортивные и развлекательные объекты': 'Прочее',
    'Иной объект': 'Прочее',

    # Ice crossing
    'Ледовая переправа': 'Ледовая переправа',
    'Ледовая переправа (официально открытая и оборудованная);': 'Ледовая переправа',
    'Стихийно возникшая (не предусмотренная) ледовая переправа': 'Ледовая переправа',

    # Absence of objects
    'Отсутствие в непосредственной близости объектов УДС и объектов притяжения': 'Нет объектов'
}

df['adj_objects'] = df['adj_objects'].replace(replace_map)
sorted(df['adj_objects'].unique())

['Дорожная инфраструктура',
 'Жилая застройка',
 'Ледовая переправа',
 'Медицина',
 'Нет объектов',
 'Образование',
 'Остановка',
 'Прочее',
 'Торговля',
 'Транспортная инфраструктура']

In [136]:
def get_objects(category):
    if category in ['Образование', 'Медицина', 'Торговля', 'Прочее']:
        return 'SOCIAL_OBJ'
    elif category in ['Транспортная инфраструктура', 'Остановка']:
        return 'TRANSPORT_OBJ'
    elif category == 'Жилая застройка':
        return 'LIVING_OBJ'
    elif category == 'Дорожная инфраструктура':
        return 'ROAD_INFRSTRCT'
    else:
        return 'no_obj'

df['adj_objects_interpretable'] = df['adj_objects'].apply(get_objects)
sorted(df['adj_objects_interpretable'].unique())

['LIVING_OBJ', 'ROAD_INFRSTRCT', 'SOCIAL_OBJ', 'TRANSPORT_OBJ', 'no_obj']

In [137]:
le = LabelEncoder()
df['adj_objects_cat'] = le.fit_transform(df['adj_objects'])

---
__cause_factors — factors influencing the traffic__

In [138]:
sorted(df['cause_factors'].unique())

['Изменение в режиме движения вследствие согласованного массового мероприятия',
 'Конструктивное сужение проезжей части вследствие уменьшения количества полос движения',
 'Наличие  пересечения (примыкания), не предусмотренного проектной документацией',
 'Наличие на проезжей части неисправного ТС, мешающего движению',
 'Наличие на проезжей части, обочине или тротуаре торгового или иного объекта, затрудняющего движение или парковку транспорта и (или) движение пешеходов',
 'Наличие на тротуарах (пешеходных дорожках, обочинах) припаркованного  транспорта,   затрудняющего движение пешеходов',
 'Наличие пересечения (примыкания), не предусмотренного проектной документацией» (не применяется с 12.2020)',
 'Нарушение режима движения вследствие несогласованного массового мероприятия',
 'Несоответствие габарита моста (путепровода) ширине проезжей части  на подходах к нему',
 "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНи

In [139]:
replace_map = {
    # Traffic disruptions due to events
    'Изменение в режиме движения вследствие согласованного массового мероприятия': 'Массовое мероприятие',
    'Нарушение режима движения вследствие несогласованного массового мероприятия': 'Массовое мероприятие',
    'Режим движения изменён вследствие проведения спецмероприятия': 'Массовое мероприятие',

    # Road accident as a cause
    'Режим движения нарушен вследствие ранее произошедшего ДТП': 'ДТП ранее',

    # Road works/narrowing
    'Проведение дорожных и прочих работ без сужения проезжей части': 'Работы без сужения',
    'Сужение проезжей части вследствие проведения работ': 'Сужение из-за работ',
    'Сужение проезжей части припаркованным транспортом': 'Сужение из-за транспорта',
    'Конструктивное сужение проезжей части вследствие уменьшения количества полос движения': 'Конструктивное сужение',

    # Work on sidewalks/pedestrians
    'Проведение работ на тротуарах (пешеходных дорожках, обочинах),  затрудняющих движение пешеходов': 'Работы на тротуарах',
    'Неудовлетворительное состояние тротуара или пешеходной дорожки (в  т.ч. недостатки зимнего содержания), затрудняющие или препятствующие движению пешеходов': 'Проблемы на тротуаре',

    # Defective/abandoned vehicle
    'Наличие на проезжей части неисправного ТС, мешающего движению': 'ТС мешает движению',
    'Наличие на тротуарах (пешеходных дорожках, обочинах) припаркованного  транспорта,   затрудняющего движение пешеходов': 'ТС мешает движению',
    
    # Foreign objects
    'Наличие на проезжей части, обочине или тротуаре торгового или иного объекта, затрудняющего движение или парковку транспорта и (или) движение пешеходов': 'Посторонние объекты',
    'Оказание влияния на движение транспортного потока или отдельного транспортного средства, наличия или внезапного возникновения на проезжей части посторонних предметов (упавшее дерево или мачта освещения, осыпь камней, селевой вынос, груз с другого транспортного сред-ства или конструктивный элемент другого транспортного средства и т.д.)': 'Посторонние объекты',
    'Оказание влияния на движение транспортного потока или отдельного транспортного средства, наличия или внезапного появления на проезжей части животных': 'Животные/препятствия',

    # Intersections, junctions
    'Наличие  пересечения (примыкания), не предусмотренного проектной документацией': 'Нелегальное примыкание',
    'Наличие пересечения (примыкания), не предусмотренного проектной документацией» (не применяется с 12.2020)': 'Нелегальное примыкание',

    # Road non-compliance with standards
    "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНиП 2.05.02.-85 'Автомобильные дороги')": 'Несоответствие категории',
    "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНиП 2.05.02.-85 ?Автомобильные дороги?)": 'Несоответствие категории',
    'Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СП 34.13330.2012 «Свод правил. Автомобильные дороги. Актуализированная редакция СНиП 2.05.02-85*»)': 'Несоответствие категории',
    'Несоответствие параметров дороги, в т.ч. геометрических, её категории на месте ДТП или на подходах к нему (величина радиуса поворота, ширина полос движения, обочин, крутизна откосов насыпи, отсутствие разделительной полосы, отсутствие виража и т.д.)': 'Несоответствие категории',
    'Несоответствие габарита моста (путепровода) ширине проезжей части  на подходах к нему': 'Несоответствие габаритов',

    # Electricity supply
    'Отключение электроснабжения на данном элементе УДС (улице, дороге, площади, дворе и т.д.) в целом': 'Отключение электроснабжения',
    'Отключение электроснабжения на данном элементе УДС (улице, дороге, площади, дворе и т.д.) в целом (не применяется с 12.2020)': 'Отключение электроснабжения',
    'Отключение электроснабжения целиком или частично в муниципальном образовании (районе, городе, районе города, посёлке, селе, деревне, ауле и т.д.)': 'Отключение электроснабжения',
    'Отключение электроснабжения целиком или частично в муниципальном образовании  (районе, городе, районе  города, посёлке, селе, деревне, ауле и т.д.)': 'Отключение электроснабжения',

    # Traffic lanes
    'Отсутствие дополнительных полос для движения (в т.ч. переходно-скоростных полос) в необходимых местах': 'Отсутствие полос',
    'Отсутствие дополнительных полос для движения (в т.ч. переходно-скоростных полос) в необходимых местах (не применяется с 12.2020)': 'Отсутствие полос',

    # Traffic lights
    "Работа светофора на регулируемом перекрёстке или регулируемом пешеходном переходе в режиме 'жёлтого мигания'": 'Режим светофора',

    # Speed ​​cameras
    'Участок, контролируемый камерами автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',
    'Участок, контролируемый стационарными камерами автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',
    'Участок, контролируемый стационарными камерами автоматической фотовидеофиксации нарушений ПДД, не обозначенный соответствующим предупреждающим знаком': 'Камера (не обозначена)',
    'Участок, контролируемый мобильными автоматической фотовидеофиксации нарушений ПДД': 'Камера (мобильная)',
    'Участок, контролируемый передвижными средствами автоматической фотовидеофиксации нарушений ПДД': 'Камера (мобильная)',
    'Участок, контролируемый камерой автоматической фотовидеофиксации нарушений ПДД, не обозначенный соответствующим предупреждающим знаком': 'Камера (не обозначена)',

    'Участок, контролируемый камерой автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',

    # Speed ​​bumps
    'Участок, оборудованный искусственными неровностями': 'Искусственные неровности',
    'Участок, оборудованный искусственными неровностями и обозначенный соответствующими дорожными знаками и разметкой': 'Искусственные неровности',
    'Участок, оборудованный искусственными неровностями, не обозначенный соответствующими дорожными знаками и (или) разметкой': 'Искусственные неровности',
    'Участок, оборудованный шумовыми полосами': 'Шумовые полосы',

    # Other
    'Сведения отсутствуют': 'Сведения отсутствуют'
}

df['cause_factors'] = df['cause_factors'].replace(replace_map)
sorted(df['cause_factors'].unique())

['ДТП ранее',
 'Животные/препятствия',
 'Искусственные неровности',
 'Камера (мобильная)',
 'Камера (не обозначена)',
 'Камера (обозначена)',
 'Конструктивное сужение',
 'Массовое мероприятие',
 'Нелегальное примыкание',
 'Несоответствие габаритов',
 'Несоответствие категории',
 'Отключение электроснабжения',
 'Отсутствие полос',
 'Посторонние объекты',
 'Проблемы на тротуаре',
 'Работы без сужения',
 'Работы на тротуарах',
 'Режим светофора',
 'Сведения отсутствуют',
 'Сужение из-за работ',
 'Сужение из-за транспорта',
 'ТС мешает движению',
 'Шумовые полосы']

In [140]:
le = LabelEncoder()
df['cause_factors_cat'] = le.fit_transform(df['cause_factors'])

---

__violations__

It's a multiclass categorial feature.

In [141]:
df['violations'] = df['violations'].apply(ast.literal_eval)       # from string "{}" to real set {}

all_violations = set()

for violations in df['violations']:
    all_violations.update(violations)

unique_violations = sorted(all_violations)
for i, v in enumerate(unique_violations, 1):
    print(f"{i}. {v}")

1. Выезд на полосу встречного движения
2. Выезд на полосу встречного движения в местах, где это запрещено
3. Выезд на полосу встречного движения с разворотом, поворотом налево или объездом препятствия
4. Выезд на трамвайные пути встречного направления
5. Движение вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины
6. Движение вдоль проезжей части при наличии и удовлетворительном состоянии тротуара
7. Движение во встречном направлении по дороге с односторонним движением
8. Другие нарушения ПДД водителем
9. Другие нарушения ПДД водителями (не применяется с 12.2020)
10. Игра на проезжей части
11. Иные нарушения
12. Нарушение водителем правил применения ремней безопасности (ставится в случае, когда не пристегнут водитель)
13. Нарушение водителем правил применения ремней безопасности (ставится в случае, когда не пристегнут пассажир)
14. Нарушение правил буксировки
15. Нарушение правил движения в жилых зонах
16. Нарушение правил движения

In [142]:
replace_map = {
    # Нарушения, связанные с движением по встречной / запрещённой полосе
    "Выезд на полосу встречного движения": "wrong_way",
    "Выезд на полосу встречного движения в местах, где это запрещено": "wrong_way",
    "Выезд на полосу встречного движения с разворотом, поворотом налево или объездом препятствия": "wrong_way",
    "Выезд на трамвайные пути встречного направления": "wrong_way",
    "Движение во встречном направлении по дороге с односторонним движением": "wrong_way",
    "Нарушение правил расположения ТС на проезжей части": "wrong_way",
    "Выезд на полосу встречного движения": "wrong_way",
    "Выезд на трамвайные пути встречного направления": "wrong_way",

    # Нарушения ПДД пешеходами
    "Пешеход в возрасте до 7 лет без взрослого": "pedestrian_violation",
    "Пешеход в состоянии алкогольного опьянения": "pedestrian_violation",
    "Пешеход в состоянии наркотического опьянения": "pedestrian_violation",
    "Нахождение на проезжей части без цели её перехода": "pedestrian_violation",
    "Игра на проезжей части": "pedestrian_violation",
    "Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода": "pedestrian_violation",
    "Переход через проезжую часть в неустановленном месте (при наличии в зоне видимости перекрёстка)": "pedestrian_violation",
    "Переход проезжей части перед приближающимся ТС с включёнными проблесковым маячком синего (сине-красного) цвета и звуковым сигналом": "pedestrian_violation",
    "Переход проезжей части в запрещённом месте (оборудованном пешеходными  ограждениями)": "pedestrian_violation",
    "Движение вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины": "pedestrian_violation",
    "Движение вдоль проезжей части при наличии и удовлетворительном состоянии тротуара": "pedestrian_violation",
    "Передвижение по проезжей части на скейтборде (роликовых коньках и т.д.)": "pedestrian_violation",
    "Пересечение велосипедистом проезжей части по пешеходному переходу": "pedestrian_violation",
    "Ходьба вдоль проезжей части при наличии и удовлетворительном состоянии тротуара": "pedestrian_violation",
    "Ходьба вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины": "pedestrian_violation",
    "Ожидание маршрутного ТС на проезжей части, вне посадочных площадок, тротуаров и обочин (при их наличии)": "pedestrian_violation",

    # Алкоголь, наркотики и отказ от освидетельствования
    "Управление ТС в состоянии алкогольного опьянения": "impaired_driving",
    "Управление ТС в состоянии наркотического опьянения": "impaired_driving",
    "Передача управления ТС лицу, находящемуся в состоянии опьянения": "impaired_driving",
    "Управление ТС лицом, находящимся в состоянии алкогольного опьянения и не имеющим права управления ТС либо лишенным права управления ТС": "impaired_driving",
    "Управление ТС лицом, находящимся в состоянии алкогольного опьянения и не имеющим права управления ТС либо лишенным права управления ТС (не применяется с 12.2020)": "impaired_driving",
    "Управление ТС лицом, находящимся в состоянии наркотического опьянения и не имеющим права управления ТС либо лишенным права управления ТС": "impaired_driving",
    "Управление ТС лицом, находящимся в состоянии наркотического опьянения и не имеющим права управления ТС либо лишенным права управления ТС (не применяется с 12.2020)": "impaired_driving",
    "Отказ водителя от прохождения медицинского освидетельствования на состояние опьянения": "impaired_driving",
    "Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС от прохождении мед. освидетельствования": "impaired_driving",
    "Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС, от прохождения мед. освидетельствования (не применяется с 12.2020)": "impaired_driving",
    "Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС от прохождении медицинского освидетельствования на состояние опьянения": "impaired_driving",
    "Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС, от прохождения медицинского освидетельствования на состояние опьянения (не применяется с 12.2020)": "impaired_driving",
    "Употребление водителем алкогольных напитков, наркотических, психотропных или иных одурманивающих веществ после дорожно-транспортного происшествия, к которому он причастен, до проведения освидетельствования с целью установления состояния опьянения или до принятия решения об освобождении от проведения такого освидетельствовани": "impaired_driving",

    # Нарушения, связанные с обгоном, перестроением, круговым движением
    "Нарушение правил обгона": "maneuver_violation",
    "Нарушение правил перестроения": "maneuver_violation",
    "Нарушение правил при круговом движении": "maneuver_violation",
    "Нарушение правил применения сигналов поворота": "maneuver_violation",
    "Разворот в местах, где разворот запрещён": "maneuver_violation",
    "Неправильный выбор дистанции": "maneuver_violation",
    "Несоблюдение бокового интервала": "maneuver_violation",
    "Неподача или неправильная подача сигналов": "maneuver_violation",
    "Неподача или неправильная подача сигналов поворота": "maneuver_violation",
    "Несоблюдение очередности проезда": "maneuver_violation",
    "Несоблюдение очередности проезда перекрестков": "maneuver_violation",
    "Несоблюдение условий, разрешающих движение транспорта задним ходом": "maneuver_violation",
    "Опасное вождение": "maneuver_violation",
    "Невыполнение требований обеспечения безопасности при начале движения": "maneuver_violation",

    # Нарушения при проезде регулируемых участков
    "Нарушение требований сигналов светофора": "traffic_control_violation",
    "Нарушение требований сигналов регулировщика": "traffic_control_violation",
    "Нарушение требований дорожных знаков": "traffic_control_violation",
    "Нарушение требований линий разметки": "traffic_control_violation",
    "Неподчинение сигналам регулирования": "traffic_control_violation",
    "Превышение установленной скорости движения": "traffic_control_violation",
    "Несоответствие скорости конкретным условиям движения": "traffic_control_violation",
    "Непредоставление преимущества в движении ТС со спецсигналами": "traffic_control_violation",

    # Отсутствие документов, прав, категории
    "Управление ТС лицом, не имеющим права на управление ТС": "license_violation",
    "Управление ТС лицом, лишенным права управления": "license_violation",
    "Управление ТС лицом, не имеющим соответствующей категории на управление ТС данного вида": "license_violation",
    "Управление мопедом лицом моложе 16 лет": "license_violation",
    "Управление велосипедом, гужевой повозкой лицом моложе 14 лет": "license_violation",
    "Управление при движении по автодороге велосипедом, гужевой повозкой лицом моложе 14 лет": "license_violation",

    # Нарушения перевозки пассажиров, детей, грузов
    "Нарушение правил перевозки людей": "transport_violation",
    "Нарушение правил перевозки детей (не использование детских сидений либо удерживающих устройств)": "transport_violation",
    "Нарушение правил перевозки опасных грузов": "transport_violation",
    "Нарушение правил перевозки крупногабаритных и тяжеловесных грузов": "transport_violation",
    "Нарушение правил применения ремней безопасности пассажиром": "transport_violation",
    "Нарушение правил применения ремней безопасности (ставится в случае, когда не пристегнут водитель)": "transport_violation",
    "Нарушение водителем правил применения ремней безопасности (ставится в случае, когда не пристегнут водитель)": "transport_violation",
    "Нарушение правил применения ремней безопасности (ставится в случае, когда не пристегнут пассажир)": "transport_violation",
    "Нарушение водителем правил применения ремней безопасности (ставится в случае, когда не пристегнут пассажир)": "transport_violation",
    "Нарушение водителем правил применения ремней безопасности (водитель)": "transport_violation",
    "Нарушение водителем правил применения ремней безопасности (пассажир)": "transport_violation",
    "Нарушение правил применения мотошлема": "transport_violation",
    "Нарушение правил применения мотошлема пассажиром": "transport_violation",
    "Нарушение правил движения тяжеловесного и(или) крупногабаритного ТС(превышение допустимых габаритов ТС, превышением допустимой массы ТС или допустимой нагрузки на ось ТС)": "transport_violation",
    "Нарушение правил движения тяжеловесного и(или) крупногабаритного ТС(превышение допустимых габаритов ТС, превышением допустимой массы ТС или допустимой нагрузки на ось ТС)": "transport_violation",
    "Нарушение правил буксировки": "transport_violation",
    "Нарушение правил погрузки, выгрузки и крепления грузов": "transport_violation",

    # Нарушения, связанные с пешеходными переходами, ЖД переездами
    "Нарушение правил проезда пешеходного перехода": "crossing_violation",
    "Нарушение правил проезда ж/д переездов": "crossing_violation",
    "Нарушение правил проезда остановок трамвая": "crossing_violation",
    "Непредоставление преимущества в движении пешеходу": "crossing_violation",

    # Нарушения освещения и технического состояния ТС
    "Эксплуатация ТС с техническими неисправностями, при которых запрещается их эксплуатация": "vehicle_tech_violation",
    "Управление транспортным средством при наличии неисправностей или условий, при которых эксплуатация транспортного средства запрещена": "vehicle_tech_violation",
    "Управление велосипедом с неисправной тормозной системой": "vehicle_tech_violation",
    "Эксплуатация технически неисправного ТС (в случае ДТП)": "vehicle_tech_violation",
    "Эксплуатация технически неисправного ТС (проставляется в случае ДТП по причине технеисправности)": "vehicle_tech_violation",

    # Прочее
    "Пользование мобильной связью во время управления автомобилем": "interference_violation",
    "Управление ТС без очков (контактных линз) в случае, когда их использование водителем обязательно": "interference_violation",
    "Создание помех для водителя в управлении транспортным средством": "interference_violation",
    "Ослепление светом фар": "interference_violation",
    "Управление ТС в состоянии нетрудоспособности (болезни)": "interference_violation",
    "Управление в состоянии переутомления, сон за рулём": "interference_violation",

    "Неожиданный выход из-за ТС": "sudden_appearance_violation",
    "Неожиданный выход из-за стоящего ТС": "sudden_appearance_violation",
    "Неожиданный выход из-за сооружений (деревьев)": "sudden_appearance_violation",
    "Оставление движущегося транспортного средства (выход или выпрыгивание на ходу и т.д.)": "sudden_appearance_violation",
    "Управление велосипедом, не оснащённым светоотражающими элементами": "sudden_appearance_violation",
    "Отсутствие световозвращающих жилетов (снаряжения) у лиц, осуществляющих работу на проезжей части": "sudden_appearance_violation",
    "Отсутствие световозвращающих элементов": "sudden_appearance_violation",
    "Стоянка на проезжей части или обочине без освещения": "sudden_appearance_violation",

    "Нарушение правил пользования внешними световыми приборами": "other_violation",
    "Нарушение правил учебной езды": "other_violation",
    "Нарушение правил остановки и стоянки": "other_violation",
    "Неповиновение или сопротивление находящемуся при исполнении служебных обязанностей сотруднику правоохранительных органов или военнослужащему при остановке или задержании ТС": "other_violation",
    "Нарушение режима труда и отдыха водителя": "other_violation",
    "Незаконное нанесение специальных цветографических схем автомобилей оперативных служб": "other_violation",
    "Нарушение правил движения в жилых зонах": "other_violation",
    "Другие нарушения ПДД водителем": "other_violation",
    "Эксплуатация незарегистрированного ТС": "other_violation",
    "Другие нарушения ПДД водителями (не применяется с 12.2020)": "other_violation",
    "Оставление места ДТП": "other_violation",
    "Светопропускание стекол менее нормативного": "other_violation",
    "Несоблюдение требований ОСАГО": "other_violation",
    "Отсутствие у водителя документов, предусмотренных законодательными и иными НПА": "other_violation",
    "Иные нарушения": "other_violation",
    "Нарушение правил пользования общественным транспортом": "other_violation",
    "Пользование во время движения незаконно установленными устройствами для подачи специальных световых и звуковых сигналов": "other_violation",
    "Непредоставление преимущества в движении ТС, имеющему нанесенные на наружные поверхности специальные цветографические схемы, надписи и обозначения, с одновременно включённым проблесковым маячком синего цвета и специальным звуковым сигналом": "other_violation"
}

def map_violation_set(violation_set):
    return set(replace_map.get(v, v) for v in violation_set)

df['violations'] = df['violations'].apply(map_violation_set)

In [143]:
all_violations = set()

for violations in df['violations']:
    all_violations.update(violations)

unique_violations = sorted(all_violations)
for i, v in enumerate(unique_violations, 1):
    print(f"{i}. {v}")

1. crossing_violation
2. impaired_driving
3. interference_violation
4. license_violation
5. maneuver_violation
6. other_violation
7. pedestrian_violation
8. sudden_appearance_violation
9. traffic_control_violation
10. transport_violation
11. vehicle_tech_violation
12. wrong_way


In [144]:
print(f"Previous num of features: {df.shape[1]}\n")

mlb = MultiLabelBinarizer()
violations_encoded = pd.DataFrame(
    mlb.fit_transform(df['violations']),
    columns=mlb.classes_,
    index=df.index
)

df = pd.concat([df, violations_encoded], axis=1)

print(f"Current num of features: {df.shape[1]}")

Previous num of features: 72

Current num of features: 84


---

__lighting__

In [145]:
sorted(df['lighting'].unique())

['В темное время суток, освещение включено',
 'В темное время суток, освещение не включено',
 'В темное время суток, освещение отсутствует',
 'Не установлено',
 'Светлое время суток',
 'Сумерки']

In [146]:
def get_lighting(category):
    if category == 'В темное время суток, освещение отсутствует':
        return 1
    else:
        return 0

df['no_lighting'] = df['lighting'].apply(get_lighting)
sorted(df['no_lighting'].unique())

[0, 1]

In [147]:
replace_map = {
    'Светлое время суток': 'День',
    'Сумерки': 'Сумерки',
    'В темное время суток, освещение включено': 'Ночь с освещением',
    'В темное время суток, освещение не включено': 'Ночь без освещения',
    'В темное время суток, освещение отсутствует': 'Ночь без освещения',
    'Не установлено': 'Не указано'
}

df['lighting'] = df['lighting'].replace(replace_map)
sorted(df['lighting'].unique())

['День', 'Не указано', 'Ночь без освещения', 'Ночь с освещением', 'Сумерки']

In [148]:
le = LabelEncoder()
df['lighting_cat'] = le.fit_transform(df['lighting'])

---

__site_objects — road network objects at the scene of RTA__

In [149]:
sorted(df['site_objects'].unique())

['АЗС',
 'Автостоянка (не отделённая от проезжей части)',
 'Автостоянка (отделенная от проезжей части)',
 'Внутридворовая территория',
 'Выезд с прилегающей территории',
 'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)',
 'Иное место',
 'Ледовая переправа',
 'Ледовая переправа(официально открытая и оборудованная)',
 'Место для перегона скота',
 'Мост, эстакада, путепровод',
 'Нерегулируемое пересечение с круговым движением',
 'Нерегулируемый ж/д переезд',
 'Нерегулируемый перекрёсток неравнозначных улиц (дорог)',
 'Нерегулируемый перекрёсток равнозначных улиц (дорог)',
 'Нерегулируемый пешеходный переход',
 'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения',
 'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации',
 'Остановка маршрутного такси',
 

In [150]:
replace_map = {
    # Pedestrian crossings
    'Нерегулируемый пешеходный переход': 'Пешеходный переход (нерегулируемый)',
    'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения': 'Пешеходный переход (нерегулируемый)',
    'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации': 'Пешеходный переход (нерегулируемый)',

    'Регулируемый пешеходный переход': 'Пешеходный переход (регулируемый)',
    'Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения': 'Пешеходный переход (регулируемый)',
    'Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации': 'Пешеходный переход (регулируемый)',

    # Railway crossings
    'Регулируемый ж/д переезд с дежурным': 'Ж/д переезд (регулируемый)',
    'Регулируемый ж/д переезд без дежурного': 'Ж/д переезд (регулируемый)',

    'Нерегулируемый ж/д переезд': 'Ж/д переезд (нерегулируемый)',

    # Crossroads
    'Регулируемый перекресток': 'Перекрёсток (регулируемый)',

    'Нерегулируемый перекрёсток неравнозначных улиц (дорог)': 'Перекрёсток (нерегулируемый)',
    'Нерегулируемый перекрёсток равнозначных улиц (дорог)': 'Перекрёсток (нерегулируемый)',

    'Нерегулируемое пересечение с круговым движением': 'Перекрёсток (круговое)',

    # Public transport
    'Остановка маршрутного такси': 'Остановка',
    'Остановка общественного транспорта': 'Остановка',
    'Остановка трамвая': 'Остановка',

    # Ice crossing
    'Ледовая переправа': 'Ледовая переправа',
    'Ледовая переправа(официально открытая и оборудованная)': 'Ледовая переправа',
    'Стихийно возникшая (не предусмотренная) ледовая переправа': 'Ледовая переправа',

    # Bridges and tunnels
    'Мост, эстакада, путепровод': 'Мост/эстакада/тоннель',
    'Подход к мосту, эстакаде, путепроводу': 'Мост/эстакада/тоннель',
    'Тоннель': 'Мост/эстакада/тоннель',

    # Parking, parking lot, adjacent territory
    'Автостоянка (отделенная от проезжей части)': 'Стоянка',
    'Автостоянка (не отделённая от проезжей части)': 'Стоянка',
    'АЗС': 'Стоянка',
    'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)': 'Стоянка',
    
    'Выезд с прилегающей территории': 'Прилегающая территория',
    'Внутридворовая территория': 'Прилегающая территория',

    # Pedestrian areas
    'Пешеходная зона': 'Пешеходная зона',
    'Тротуар, пешеходная дорожка': 'Пешеходная зона',

    # Other
    'Место для перегона скота': 'Прочее',
    'Перегон (нет объектов на месте ДТП)': 'Прочее',
    'Иное место': 'Прочее'
}


df['site_objects'] = df['site_objects'].replace(replace_map)
sorted(df['site_objects'].unique())

['Ж/д переезд (нерегулируемый)',
 'Ж/д переезд (регулируемый)',
 'Ледовая переправа',
 'Мост/эстакада/тоннель',
 'Остановка',
 'Перекрёсток (круговое)',
 'Перекрёсток (нерегулируемый)',
 'Перекрёсток (регулируемый)',
 'Пешеходная зона',
 'Пешеходный переход (нерегулируемый)',
 'Пешеходный переход (регулируемый)',
 'Прилегающая территория',
 'Прочее',
 'Стоянка']

In [151]:
le = LabelEncoder()
df['site_objects_cat'] = le.fit_transform(df['site_objects'])

---

__injury_severity__

Also has multiple classes.

In [152]:
df['injury_severity'] = df['injury_severity'].apply(ast.literal_eval)       # from string "{}" to real set {}

all_harms = set()

for harm in df['injury_severity']:
    all_harms.update(harm)

unique_harms = sorted(all_harms)
for i, v in enumerate(unique_harms, 1):
    print(f"{i}. {v}")

1. Не пострадал
2. Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)
3. Получил телесные повреждения с показанием к лечению в медицинских организациях, фактически лечение не проходил, к категории раненый не относится
4. Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится
5. Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)
6. Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара
7. Раненый, находящийся (находившийся) на стационарном лечении
8. Скончался в течение 1 суток
9. Скончался в течение 10 суток
10. Скончался в течение 11 суток
11. Скончался в течение 12 суток
12. Скончался в течение 13 суток
13. Скончался в течение 14 суток
14. Скончался в течение 15 суток
15. Скончался в течение 16 суто

In [153]:
replace_map = {
    'Не пострадал': 'minor_injury',
    'Получил телесные повреждения с показанием к лечению в медицинских организациях, фактически лечение не проходил, к категории раненый не относится': 'minor_injury',
    'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится': 'minor_injury',

    # Moderate injuries (not hospitalization)
    'Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)': 'injury_no_hospitalization',
    'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)': 'injury_no_hospitalization',
    'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара': 'injury_no_hospitalization',

    # Severe injuries (hospitalization)
    'Раненый, находящийся (находившийся) на стационарном лечении': 'injury_with_hospitalization',

    # Death on the spot
    'Скончался на месте ДТП до приезда скорой медицинской помощи': 'death_before_hospital',
    'Скончался на месте ДТП по прибытию скорой медицинской помощи, но до транспортировки в мед. организацию': 'death_before_hospital',
    'Скончался на месте ДТП по прибытию скорой медицинской помощи, но до транспортировки в медицинское учреждение': 'death_before_hospital',

    # Death during transportation
    'Скончался при транспортировке': 'death_before_hospital',

    # Death
    'Скончался в течение 1 суток': 'death_within_30_days',
    'Скончался в течение 2 суток': 'death_within_30_days',
    'Скончался в течение 3 суток': 'death_within_30_days',
    'Скончался в течение 4 суток': 'death_within_30_days',
    'Скончался в течение 5 суток': 'death_within_30_days',
    'Скончался в течение 6 суток': 'death_within_30_days',
    'Скончался в течение 7 суток': 'death_within_30_days',
    'Скончался в течение 8 суток': 'death_within_30_days',
    'Скончался в течение 9 суток': 'death_within_30_days',
    'Скончался в течение 10 суток': 'death_within_30_days',
    'Скончался в течение 11 суток': 'death_within_30_days',
    'Скончался в течение 12 суток': 'death_within_30_days',
    'Скончался в течение 13 суток': 'death_within_30_days',
    'Скончался в течение 14 суток': 'death_within_30_days',
    'Скончался в течение 15 суток': 'death_within_30_days',
    'Скончался в течение 16 суток': 'death_within_30_days',
    'Скончался в течение 17 суток': 'death_within_30_days',
    'Скончался в течение 18 суток': 'death_within_30_days',
    'Скончался в течение 19 суток': 'death_within_30_days',
    'Скончался в течение 20 суток': 'death_within_30_days',
    'Скончался в течение 21 суток': 'death_within_30_days',
    'Скончался в течение 22 суток': 'death_within_30_days',
    'Скончался в течение 23 суток': 'death_within_30_days',
    'Скончался в течение 24 суток': 'death_within_30_days',
    'Скончался в течение 25 суток': 'death_within_30_days',
    'Скончался в течение 26 суток': 'death_within_30_days',
    'Скончался в течение 27 суток': 'death_within_30_days',
    'Скончался в течение 28 суток': 'death_within_30_days',
    'Скончался в течение 29 суток': 'death_within_30_days',
    'Скончался в течение 30 суток': 'death_within_30_days',
}

def map_violation_set(violation_set):
    return set(replace_map.get(v, v) for v in violation_set)

df['injury_severity'] = df['injury_severity'].apply(map_violation_set)

In [154]:
all_harms = set()

for harm in df['injury_severity']:
    all_harms.update(harm)

unique_harms = sorted(all_harms)
for i, v in enumerate(unique_harms, 1):
    print(f"{i}. {v}")

1. death_before_hospital
2. death_within_30_days
3. injury_no_hospitalization
4. injury_with_hospitalization
5. minor_injury


In [155]:
# print(f"Previous num of features: {df.shape[1]}\n")

# mlb = MultiLabelBinarizer()
# harms_encoded = pd.DataFrame(
#     mlb.fit_transform(df['bodily harm']),
#     columns=mlb.classes_,
#     index=df.index
# )

# df = pd.concat([df, harms_encoded], axis=1)

# print(f"Current num of features: {df.shape[1]}")

severity_rank = {
    'minor_injury': 0,
    'injury_no_hospitalization': 1,
    'injury_with_hospitalization': 2,
    'death_before_hospital': 3,
    'death_within_30_days': 3
}

df['injury_grouped'] = df['injury_severity'].apply(
    lambda x: {replace_map.get(i, i) for i in x} if isinstance(x, (set, list)) else set()
)

df['severity'] = df['injury_grouped'].apply(
    lambda group: max((severity_rank.get(item, -1) for item in group), default=-1)
)

drop_columns.append('injury_grouped')

In [156]:
df['severity'].value_counts()

severity
1    210028
2    185577
3     77583
0         6
Name: count, dtype: int64

In [157]:
df = df[df['severity'] != 0]
# because of lack of obs

---

In [158]:
# drop raw categorial features
drop_columns.extend(cat_cols)

## Create some new features

In [159]:
df['YEAR'] = pd.to_datetime(df['DATE']).dt.year
df['MONTH'] = pd.to_datetime(df['DATE']).dt.month
df['WEEKDAY'] = pd.to_datetime(df['DATE']).dt.weekday  # 0 = Mon, 6 = Sun
df['SEASON'] = df['MONTH'] % 12 // 3 + 1
df['is_WEEKEND'] = df['WEEKDAY'].isin([5, 6]).astype(int)

C:\Users\Настя\AppData\Local\Temp\ipykernel_22752\3878303528.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['YEAR'] = pd.to_datetime(df['DATE']).dt.year
C:\Users\Настя\AppData\Local\Temp\ipykernel_22752\3878303528.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['MONTH'] = pd.to_datetime(df['DATE']).dt.month
C:\Users\Настя\AppData\Local\Temp\ipykernel_22752\3878303528.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['WEEKDAY'] = pd.to_datetime(df['DATE']).dt.weekday  # 0 = Mon, 6 = Sun


In [160]:
df['HOUR'] = pd.to_datetime(df['TIME'], format='%H:%M').dt.hour

df['is_NIGHT'] = df['HOUR'].apply(lambda x: 1 if (x < 6 or x >= 22) else 0)

df['is_PEAK_HOUR'] = df['HOUR'].apply(lambda x: 1 if x in [7,8,9,16,17,18] else 0)

In [161]:
drop_columns.extend(['TIME'])

## Drop unnecessary columns

In [162]:
print(drop_columns)

['road_km', 'road_m', 'NP', 'injury_grouped', 'injury_severity', 'cause_factors', 'street_rank', 'road_defects', 'TYPE', 'traffic_changes', 'site_objects', 'violations', 'weather', 'road_rank', 'road_surface', 'lighting', 'adj_objects', 'TIME']


In [163]:
df = df.drop(drop_columns, axis=1)

print(f"Features left: {df.shape[1]}, in the begining was: {features_number}")

Features left: 79, in the begining was: 59


In [164]:
df.columns

Index(['REGION', 'DATE', 'COORD_L', 'COORD_W', 'road_name', 'road_category',
       'n_VEHICLES', 'n_PARTICIPANTS', 'ID', 'n_DEATHS', 'n_INJURED',
       'vehicle_failure', 'non_private_vehicle', 'russian_vehicle',
       'white_vehicle', 'black_vehicle', 'colored_vehicle', 'drunk_driver',
       'female_driver', 'escaped', 'no_seatbelt_injury', 'n_drunk',
       'n_children', 'n_cyclists', 'n_pedestrians', 'vehicle_age_min',
       'vehicle_age_max', 'vehicle_age_avg', 'n_class_a', 'n_class_b',
       'n_class_c', 'n_class_d', 'n_class_e', 'n_class_s', 'n_front_drive',
       'n_rear_drive', 'n_4wd', 'n_guilty', 'guilty_share',
       'n_fatal_violations', 'guilty_exp_avg', 'exp_avg', 'road_rank_cat',
       'road_defects_cat', 'traffic_changes_bin', 'traffic_changes_cat',
       'road_surface_cat', 'TYPE_cat', 'out_of_town', 'street_rank_cat',
       'weather_interpretable', 'weather_cat', 'adj_objects_interpretable',
       'adj_objects_cat', 'cause_factors_cat', 'crossing_violation

## Processing the data gaps (part 2) with ML algorithms

After initial processing of gaps there are still columns with NaN. In this part we will fill them using machine learning methods.

In [165]:
cols_with_gaps = df.columns[df.isna().any()].tolist()

print('Columns for proceed:\n')
for col in cols_with_gaps:
    gaps = df[col].isna().sum()
    print(f"'{col}' — {gaps} gaps")

print(f"\nCurrent num of obs: {df.shape[0]}")

Columns for proceed:

'road_category' — 11652 gaps
'vehicle_age_min' — 6207 gaps
'vehicle_age_max' — 6207 gaps
'vehicle_age_avg' — 6207 gaps

Current num of obs: 473188


In [166]:
df_ml = df.copy()

# encoding 'dor'
le = LabelEncoder()
df_ml['road_name_encoded'] = le.fit_transform(df_ml['road_name'])

In [167]:
# train models to dill the gaps on these data
df_train = df_ml.dropna()
print(f"Num of obs in train DataFrame: {df_train.shape[0]}")

Num of obs in train DataFrame: 455563


#### **'road_category'**

In [168]:
df['road_category'].unique()

array([ 5.,  6.,  7.,  3.,  4.,  2.,  1.,  8., nan])

Analysis shows that `'road_category'` is a categorical preproceed feature.

In [169]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

df_knn = df_train[['road_name_encoded', 'road_rank_cat', 'COORD_L', 'COORD_W', 'road_category']]

X = df_knn[['road_name_encoded', 'road_rank_cat', 'COORD_L', 'COORD_W']]
y = df_knn['road_category'].astype(int)

# normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_val)

print("Classification report:\n", classification_report(y_val, y_pred))

Classification report:
               precision    recall  f1-score   support

           1       0.62      0.67      0.65      7249
           2       0.54      0.47      0.51      5393
           3       0.49      0.34      0.40      3721
           4       0.72      0.76      0.74     22801
           5       0.79      0.82      0.80     27332
           6       0.80      0.80      0.80     19785
           7       0.67      0.50      0.57      2172
           8       0.73      0.56      0.64      2660

    accuracy                           0.73     91113
   macro avg       0.67      0.62      0.64     91113
weighted avg       0.73      0.73      0.73     91113



In [170]:
df_missing = df_ml[df_ml['road_category'].isna()]

X_missing = df_missing[['road_name_encoded', 'road_rank_cat', 'COORD_L', 'COORD_W']]
X_missing_scaled = scaler.transform(X_missing)

predicted_dor_k = knn.predict(X_missing_scaled)

# Записываем результат обратно в основной DataFrame
df.loc[df_missing.index, 'road_category'] = predicted_dor_k

In [171]:
gaps = df['road_category'].isna().sum()
print(f"Column 'road_category' has {gaps} gaps.")

Column 'road_category' has 0 gaps.


#### **'vehicle_age_min', 'vehicle_age_max', 'vehicle_age_avg'**

In [172]:
df[['vehicle_age_min', 'vehicle_age_max', 'vehicle_age_avg']].dtypes

vehicle_age_min    float64
vehicle_age_max    float64
vehicle_age_avg    float64
dtype: object

In [173]:
features = [
    'n_VEHICLES', 'n_PARTICIPANTS', 'non_private_vehicle', 'russian_vehicle',
    'drunk_driver', 'escaped',
    'n_class_a', 'n_class_b', 'n_class_c',
    'n_class_d', 'n_class_e', 'n_class_s', 'n_front_drive',
    'n_rear_drive', 'n_4wd',
    'exp_avg', 'guilty_exp_avg',
    'vehicle_failure',
    'weather_cat', 'lighting_cat', 'cause_factors_cat', 'out_of_town'
]

X = df_train[features]
y = df_train[['vehicle_age_min', 'vehicle_age_max', 'vehicle_age_avg']].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mae_min = mean_absolute_error(y_test.iloc[:, 0], y_pred[:, 0])
mae_max = mean_absolute_error(y_test.iloc[:, 1], y_pred[:, 1])
mae_avg = mean_absolute_error(y_test.iloc[:, 2], y_pred[:, 2])

print(f"MAE (min vehicles age): {mae_min:.2f}")
print(f"MAE (max vehicles age): {mae_max:.2f}")
print(f"MAE (avg vehicles age): {mae_avg:.2f}")
print(f"\nOverall MAE (all targets averaged): {mae:.2f}")

MAE (min vehicles age): 6.44
MAE (max vehicles age): 4.98
MAE (avg vehicles age): 5.32

Overall MAE (all targets averaged): 5.58


In [174]:
df_missing = df_ml[df_ml[['vehicle_age_min', 'vehicle_age_max', 'vehicle_age_avg']].isna().any(axis=1)]
X_missing = df_missing[features]

y_pred = model.predict(X_missing)

y_pred_fixed = y_pred.copy()

for i in range(y_pred.shape[0]):
    min_val, max_val, avg_val = y_pred[i]

    min_val = min(min_val, avg_val, max_val)
    max_val = max(min_val, avg_val, max_val)
    avg_val = max(min_val, min(avg_val, max_val))

    y_pred_fixed[i] = [min_val, max_val, avg_val]


df.loc[df_missing.index, 'vehicle_age_min'] = y_pred_fixed[:, 0]
df.loc[df_missing.index, 'vehicle_age_max'] = y_pred_fixed[:, 1]
df.loc[df_missing.index, 'vehicle_age_avg'] = y_pred_fixed[:, 2]

In [175]:
gaps = df['vehicle_age_min'].isna().sum()
print(f"Column 'min vehicles age' has {gaps} gaps.")

gaps = df['vehicle_age_max'].isna().sum()
print(f"Column 'max vehicles age' has {gaps} gaps.")

gaps = df['vehicle_age_avg'].isna().sum()
print(f"Column 'avg vehicles age' has {gaps} gaps.")

Column 'min vehicles age' has 0 gaps.
Column 'max vehicles age' has 0 gaps.
Column 'avg vehicles age' has 0 gaps.


In [176]:
df.shape

(473188, 79)

In [177]:
df.to_csv('data/DTP_DATA_2025_PROCESSED.csv')